In [1]:
import pandas as pd
pd.options.display.max_columns = None # display all the columns
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# load dataset

df_original = pd.read_csv('Data_features.csv') # feature set
obs = pd.read_csv('Data_train_set.csv') # User ID, Churn
est = pd.read_csv('Data_test_set.csv')  # User ID to predict Churn
obs_id = obs['ID'].values # IDs in factual data
est_id = est['ID'].values # IDs to analyse
df = df_original[df_original.ID.isin(obs_id)] # df for training
df_lm = df[df.MONTH_NUM_FROM_EVENT == 1] # the last month data

df_blm = df[df.MONTH_NUM_FROM_EVENT == 2] # the month before the last month data

In [2]:
d = { e: '2' + e for e in list(df_blm.columns)[2:]}
df_blm_ren = df_blm.rename(index=str, columns=d)

In [3]:
# pd.concat([df_lm, df_blm.drop('ID')]
df_blm_drop = df_blm_ren.drop(['ID', 'MONTH_NUM_FROM_EVENT'], axis=1)

In [4]:
df_2 = pd.concat([df_lm.reset_index(drop=True), df_blm_drop.reset_index(drop=True)], axis=1)

In [5]:
X_1 = df_lm.drop(['ID', 'MONTH_NUM_FROM_EVENT'], axis=1).values
X_2 = np.array(df_blm_drop.values)
X_1.shape, X_2.shape

((12890, 46), (12890, 46))

In [6]:
X = np.array(df_2.values)[:,2:] # array of features from 
y = np.array(obs.TARGET)

X = np.concatenate((X_1, X_2), axis=1)
from sklearn.utils.validation import assert_all_finite
nanplaces = np.isnan(X)
X[nanplaces] = 0.01
# run checks
print(assert_all_finite(X))
# should output None
print(np.any(np.isnan(X)), np.all(np.isfinite(X))) 
# should output False True
X.shape

None
False True


(12890, 92)

In [7]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [8]:
# for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
C = 0.2
LR = LogisticRegression(C=C, solver='liblinear', max_iter=4000)
LR.fit(X_train,y_train)
y_prob_ = LR.predict_proba(X_test)[:,1]
y_ = LR.predict(X_test)
print('C =', C, 'using', 'liblinear', 'solver')
print(confusion_matrix(y_test, y_, labels=[1,0]))
print("AUC ROC score", roc_auc_score(y_test, y_prob_))

C = 0.2 using liblinear solver
[[   3  194]
 [   2 2379]]
AUC ROC score 0.7662821362862083
